<a href="https://colab.research.google.com/github/arik08/Sharing/blob/master/%5BStudy%5D_Data_Correlation_Time_delay_Analysis_Core.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
df = pd.read_excel('/content/drive/My Drive/Data_for_Analysis/Finex_Study/MG_Pressure_mins_temp.xlsx')

In [0]:
######## 데이터 산점도 Plot 출력 기능도 추가할 것. R2도 출력?
# 핵심코드...?

def corr_mean(num1,num2,delay_t=150,chartshow=False):
    window_size = 300 # 상관계수 산출 샘플 수
    t_start = 0
    t_end = t_start + window_size
    step_size = 50 # Window move steps
    rss=[]
    while t_end < 5400:
        d1 = df.iloc[t_start:t_end,num1] # CO2 농도
        d2 = df.iloc[t_start:t_end,num2] # 측정하고자 하는 태그
        rs = [d1.corr(d2.shift(lag)) for lag in range(-int(delay_t),int(delay_t+1))] # Core Process
        rss.append(rs)
        t_start = t_start + step_size
        t_end = t_end + step_size
    rss = pd.DataFrame(rss) # 2차원 Heatmap 배열 완성

    x = range(-delay_t,delay_t+1)
    rss_mean = [rss.iloc[:,i].mean() for i in range(301)] # 열 방향 값들의 평균 산출, 배열 생성
    rss_mean_cen = rss_mean[int(np.ceil(len(rss_mean)/2))] 
    
    rss_max = max(rss_mean) 
    rss_min = min(rss_mean)
    offset_max = np.ceil(len(rss_mean)/2) - np.argmax(rss_mean) 
    offset_min = np.ceil(len(rss_mean)/2) - np.argmin(rss_mean)
    
    if abs(rss_max) > abs(rss_min): 
        rss_opt = rss_max 
        offset_opt = offset_max
    else:
        rss_opt = rss_min
        offset_opt = offset_min

    rss_val = [0,0,0,0,0,0,0]
    rss_val[0] = round(rss_mean_cen,3) # 시간차 미고려시(=0) 상관계수
    rss_val[1] = round(rss_opt,3)      # 최적 상관계수 값
    rss_val[2] = int(-offset_opt)+1    # 최적 상관계수 시간차
    rss_val[3] = round(rss_max,3)      # 최대 상관계수 값
    rss_val[4] = int(offset_max)+1     # 최대 상관계수 시간차
    rss_val[5] = round(rss_min,3)      # 최소 상관계수 값
    rss_val[6] = int(offset_min)+1     # 최소 상관계수 시간차
    # rss_val[7] = ###############     # 최적 시간차에서의 시계열 표준편차

    if chartshow:
        f,ax = plt.subplots(figsize=(10,9))
        sns.heatmap(rss,cmap='RdBu_r',ax=ax)
        tag_name1=df.columns[num1]; tag_name2=df.columns[num2]
        ax.set(title=f'xCorr : ['+str(num1)+']'+str(tag_name1)+' - '+'['+str(num2)+']'+str(tag_name2)
        +', cen:'+str(round(rss_val[0],2))+', opt:'+str(round(rss_val[1],2))+', offset:'+str(round(rss_val[2],2)),xlim=[0,301], xlabel='Offset',ylabel='Epochs')
        ax.set_xticks([0, 50, 100, 151, 201, 251, 301])
        ax.set_xticklabels([-150, -100, -50, 0, 50, 100, 150]);
        plt.savefig('/content/drive/My Drive/Images/xCorr_'+str(num1)+'.'+str(num2)+'_'+str(tag_name1)+'-'+str(tag_name2)+'.png')
        plt.close()

        plt.figure(2,figsize = (10,8))
        plt.subplot(2,1,1)
        plt.plot(x,rss_mean,'b-')
        plt.grid()
        plt.axis([-150,150,-0.6,0.6])
        plt.hlines(0,-150,150,colors='r')
        plt.vlines(0,-0.6,0.6,colors='r')
        plt.title('['+str(num1)+']'+str(tag_name1)+' - '+'['+str(num2)+']'+str(tag_name2)
        +', cen:'+str(round(rss_val[0],2))+', opt:'+str(round(rss_val[1],2))+', offset:'+str(round(rss_val[2],2)))

        plt.subplot(2,1,2)
        if abs(rss_max) > abs(rss_min): 
            opt_posit = np.argmax(rss_mean)
        else:
            opt_posit = np.argmin(rss_mean)
        plt.plot(rss.iloc[:,opt_posit],'b-')
        plt.grid()
        plt.hlines(0,0,100,colors='r')
        plt.axis([0,100,-0.8,0.8])
        plt.xlabel('Time')

        plt.savefig('/content/drive/My Drive/Images/xCorr_'+str(num1)+'.'+str(num2)+'_'+str(tag_name1)+'-'+str(tag_name2)+'_line.png')
        plt.close()

    return x,rss_mean,rss_val

In [0]:
# 중간 확인용
a,b,c = corr_mean(89,114,chartshow=True)
print('Center_rs :',round(c[0],3))
print('Optimum_rs:',round(c[1],3))
print('Time shift:',int(c[2]),'steps')

Center_rs : 0.064
Optimum_rs: 0.275
Time shift: -9 steps


In [0]:
# 차트 자동저장 및 엑셀 결과 출력용

rss_mean=[]
for i in [3]: # 비교대상 (Y)
    # for j in range(1,169+1): #1~169 + 1
    for j in [4,5,6,7,8,9]: # 비교대상 (X)
        print("Calculating...",i,'-',j)
        try:
            a,b,c = corr_mean(i,j,chartshow=True)
            rss_mean.append([i,j,c[0],c[1],c[2],c[3],c[4],c[5],c[6]])
        except ValueError:  
            pass

rss_mean = pd.DataFrame(rss_mean)
rss_mean.to_csv("/content/drive/My Drive/Output/rss_mean.csv", header=False, index=False)